In [457]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import *
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pickle
import os

In [458]:
def first_nonnull_location(d):
    for i in range(d.shape[0]):
        if np.sum(np.sum(pd.DataFrame(d.values[i]).isnull())) == 0:
            return i
        
'''Function that returns the number of null values present in each row
till the end. Takes three parameters, start, end and the dataframe on 
which to apply this.
'''

def rows_null_check(start,end,d):
    i=start
    index=[]
    
    while (i<end):
        temp = np.sum(np.sum((pd.DataFrame(d.values[i]).isnull() )))
        if temp > 0:
            index.append(i)
        temp=0
        i+=1
    
    return (index)
        

In [459]:
#London Dataset.
#60-min. and 1-min.
d60 = pd.read_csv("/media/siddharth/New Volume/Github/TherML Data/Open Power System Data/opsd-household_data-2017-09-01/household_data_60min_singleindex.csv",header=0)
d60c = pd.read_csv("/media/siddharth/New Volume/Github/TherML Data/Open Power System Data/opsd-household_data-2017-09-01/household_data_60min_singleindex.csv",header=0)

d60_year = pd.DataFrame(d60c.values[2135:10920])
d60_year.columns = d60c.columns

d1 = pd.read_csv("/media/siddharth/New Volume/Github/TherML Data/Open Power System Data/opsd-household_data-2017-09-01/household_data_1min_singleindex.csv",header=0)
d3 = pd.read_csv("/media/siddharth/New Volume/Github/TherML Data/Open Power System Data/opsd-household_data-2017-09-01/household_data_3min_singleindex.csv",header=0)
d15 = pd.read_csv("/media/siddharth/New Volume/Github/TherML Data/Open Power System Data/opsd-household_data-2017-09-01/household_data_15min_singleindex.csv",header=0)

In [460]:
d1.head()

,utc_timestamp,cet_cest_timestamp,DE_KN_residential4_dishwasher,DE_KN_residential4_ev,DE_KN_residential4_freezer,DE_KN_residential4_grid_export,DE_KN_residential4_grid_import,DE_KN_residential4_heat_pump,DE_KN_residential4_pv,DE_KN_residential4_refrigerator,DE_KN_residential4_washing_machine,interpolated_values
0,2016-09-06T15:05:00Z,2016-09-06T17:05:00+0200,84.616,853.246,152.436,7748.205,3350.05,2222.02,9738.580,NaN,100.979,NaN
1,2016-09-06T15:06:00Z,2016-09-06T17:06:00+0200,84.616,853.246,152.437,7748.255,3350.05,2222.02,9738.625,NaN,100.979,NaN
2,2016-09-06T15:07:00Z,2016-09-06T17:07:00+0200,84.616,853.246,152.437,7748.364,3350.05,2222.02,9738.741,NaN,100.979,NaN
3,2016-09-06T15:08:00Z,2016-09-06T17:08:00+0200,84.616,853.246,152.438,7748.458,3350.05,2222.02,9738.841,NaN,100.979,NaN
4,2016-09-06T15:09:00Z,2016-09-06T17:09:00+0200,84.616,853.246,152.438,7748.552,3350.05,2222.02,9738.941,NaN,100.979,NaN


In [461]:
d60.head()

,utc_timestamp,cet_cest_timestamp,DE_KN_residential4_dishwasher,DE_KN_residential4_ev,DE_KN_residential4_freezer,DE_KN_residential4_grid_export,DE_KN_residential4_grid_import,DE_KN_residential4_heat_pump,DE_KN_residential4_pv,DE_KN_residential4_refrigerator,DE_KN_residential4_washing_machine,interpolated_values
0,2015-10-04T01:00:00Z,2015-10-04T03:00:00+0200,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN,NaN
1,2015-10-04T02:00:00Z,2015-10-04T04:00:00+0200,NaN,NaN,NaN,NaN,NaN,0.02,NaN,NaN,NaN,NaN
2,2015-10-04T03:00:00Z,2015-10-04T05:00:00+0200,NaN,NaN,NaN,NaN,NaN,0.03,NaN,NaN,NaN,NaN
3,2015-10-04T04:00:00Z,2015-10-04T06:00:00+0200,NaN,NaN,NaN,NaN,NaN,0.04,NaN,NaN,NaN,NaN
4,2015-10-04T05:00:00Z,2015-10-04T07:00:00+0200,NaN,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN,NaN


In [462]:
#Remove the last column.
del d1["interpolated_values"]
del d3["interpolated_values"]
del d15["interpolated_values"]
del d60["interpolated_values"]

del d1["DE_KN_residential4_grid_export"]
del d3["DE_KN_residential4_grid_export"]
del d15["DE_KN_residential4_grid_export"]
del d60["DE_KN_residential4_grid_export"]

del d1["DE_KN_residential4_pv"]
del d3["DE_KN_residential4_pv"]
del d15["DE_KN_residential4_pv"]
del d60["DE_KN_residential4_pv"]


In [463]:
for i in range(d60_refrigerator.shape[0]):
    if(d60_refrigerator[i]>0): break
print i        

1


In [464]:
d60_refrigerator = np.empty([d60_year.shape[0],1],dtype=float)
d60_dishwasher = np.empty([d60_year.shape[0],1],dtype=float)
d60_ev = np.empty([d60_year.shape[0],1],dtype=float)
d60_freezer = np.empty([d60_year.shape[0],1],dtype=float)
d60_grid_import = np.empty([d60_year.shape[0],1],dtype=float)
d60_heat_pump = np.empty([d60_year.shape[0],1],dtype=float)
d60_washing_machine = np.empty([d60_year.shape[0],1],dtype=float)

for i in range(d60_year.shape[0]):
    d60_refrigerator[i] = (0.0168 * i)
    d60_ev[i] = (0.1084 * i)
    d60_freezer[i] = (0.0182 * i) 
    d60_dishwasher[i] = (0.0122 * i) 
    d60_washing_machine[i] = (0.0122 * i) 

for i in range(-2135,9731):    
    d60_heat_pump[i] = 1*(10**(-8))*(i**3)-0.0002*(i**2) + 1.2896*i - 850.03
    
for i in range(-711,11155):    
    d60_grid_import[i] = 2*(10**(-8))*(i**3)-0.0003*(i**2) + 1.2728*i + 1063.9    

IndexError: index 8785 is out of bounds for axis 0 with size 8785

In [ ]:
d60_year["DE_KN_residential4_refrigerator"] = d60_refrigerator
d60_year["DE_KN_residential4_dishwasher"] = d60_dishwasher
d60_year["DE_KN_residential4_ev"] = d60_ev
d60_year["DE_KN_residential4_freezer"] = d60_freezer
d60_year["DE_KN_residential4_grid_import"] = d60_grid_import
d60_year["DE_KN_residential4_heat_pump"] = d60_heat_pump
d60_year["DE_KN_residential4_washing_machine"] = d60_washing_machine

###### d60.head()

In [ ]:
d60[d60["utc_timestamp"].str.contains("2017-01-01T")]

In [ ]:
d60[d60["utc_timestamp"].str.contains("2016-01-01T")]

In [ ]:
#Remove the first row from 'd60'
d60_cols = d60.columns
d60 = pd.DataFrame(d60.values[1:])
d60.columns = d60_cols

In [ ]:
d60.to_csv("NewData.csv",sep=",",encoding="utf-8")

In [ ]:
d60.values[2553][-3]

In [ ]:
d60.values[2554][-3]

In [ ]:
d60c.values[2554][-5]

In [ ]:
d60c.values[2553]

In [ ]:
d60.columns

In [ ]:
d60_year.head()

In [ ]:
d60_year.tail()

In [472]:
d60 = pd.read_csv("/media/siddharth/New Volume/Github/TherML Data/Open Power System Data/opsd-household_data-2017-09-01/household_data_60min_singleindex.csv",header=0)
del d60["interpolated_values"]

In [474]:
d60.head()

,utc_timestamp,cet_cest_timestamp,DE_KN_residential4_dishwasher,DE_KN_residential4_ev,DE_KN_residential4_freezer,DE_KN_residential4_grid_export,DE_KN_residential4_grid_import,DE_KN_residential4_heat_pump,DE_KN_residential4_pv,DE_KN_residential4_refrigerator,DE_KN_residential4_washing_machine
0,2015-10-04T01:00:00Z,2015-10-04T03:00:00+0200,NaN,NaN,NaN,NaN,NaN,0.00,NaN,NaN,NaN
1,2015-10-04T02:00:00Z,2015-10-04T04:00:00+0200,NaN,NaN,NaN,NaN,NaN,0.02,NaN,NaN,NaN
2,2015-10-04T03:00:00Z,2015-10-04T05:00:00+0200,NaN,NaN,NaN,NaN,NaN,0.03,NaN,NaN,NaN
3,2015-10-04T04:00:00Z,2015-10-04T06:00:00+0200,NaN,NaN,NaN,NaN,NaN,0.04,NaN,NaN,NaN
4,2015-10-04T05:00:00Z,2015-10-04T07:00:00+0200,NaN,NaN,NaN,NaN,NaN,0.06,NaN,NaN,NaN


In [476]:
for j in range(2,d60.columns.shape[0]):
    for i in range(d60.shape[0]):
        if( np.isnan(d60.values[i][j]) == True ):
            #If the value is NULL.
            #Find the next NULL value.
            for temp in range(i+1,d60.shape[0]):
                if( np.isnan(d60.values[temp][j]) == True):
                    break
             #Now, everything between 'i' and 'temp' should be equal to d60.values[i], if d60.values[i] == d60.values[temp].
            if(d60.values[i][j] == d60.values[temp][j]):
                for ni in range(i+1,temp):
                    d60.values[ni][j] = d60.values[i][j]
            #Else, assume that 'temp's value' is greater than 'i's value'.
            nval = temp-i-1 
            diff = d60.values[temp][j] - d60.values[i][j]
            for ni in range(i+1,temp):
                d60.values[ni][j] = (diff)/nval + d60.values[ni-1][j]

KeyboardInterrupt: 